## Intro

This notebook pulls out the key parts of the Eikon API tutorial in order to build up useful scripts in the future. This includes any modifications to the files in the tutorial, as well as code snippets and workflows.

## Setting up Eikon API

In each new environment, you will need to reinstall eikon, as well as then import it.

In [ ]:
## "eikon" is an environment create already within anaconda, but it could be something different.

conda activate eikon
pip install eikon # no need to do this if eikon already installed in the environment
import eikon as ek

# plus import any other packages needed
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp

You need to ensure that when you login to eikon, you do so without saving your key in the global environment or anywhere in your code/markdown - doing so will breach your eikon licencing agreement as this API keys are very sensitive information. Once you've saved this, you can then log in to the API.

In [ ]:
## set app_id in your local environment OR via a text file saved in the current working directory

ek.set_app_key(app_id)

## then you're in

## Setting up Refinitiv Data Platform (RDP/EDP) API

RDP is a different API to the Eikon API.

In [ ]:
conda activate eikon


## Data retrieval

We will mostly be using the API only to retrieve data from Eikon, before exporting it into R to manipulate and analyse.

### Deal screener

You can use the Excel plug-in for Eikon to create formulas (including data items) that can then be copied and pasted into the Python script.

To do this, you create the search function you want in the SCREENER app on Eikon desktop, then "export as formulas to Excel". 

An example is below.



In [4]:
## example for JBS (using its PermID only)

syntax = 'SCREEN(U(IN(DEALS)/*UNV:DEALSLOAN*/),IN(TR.LNParticipant(LNPartRole=LNBUP),4296555324))'

fields = ["TR.LNSdcDealNum,TR.LNAnnouncementDate,TR.LNTrancheClosingDatePrint,TR.LNIssuer,TR.LNIssuerPERMID,TR.LNIssuerPriSIC,TR.LNIssuerNation,TR.LNTotalFacilityAmount(Scale=6),TR.LNTrancheAmount(Scale=6),TR.LNTargetMarket,TR.LNUseOfProceeds(Concat='|'),TR.LNTrancheType,TR.LNYieldType,TR.LNInitialPricingSpread(Concat='|'),TR.LNMaturityDate,TR.LNBookRunner(Concat='|'),TR.LNPrincipalAmountPerBookRunnerThisMarket(Scale=6),TR.LNMandatedArranger(Concat='|')"]

df,e = ek.get_data(syntax, fields) # adding ,e presents the answers in a nice format
df

## checking whether you can separate instruments from syntax

instruments = '4296555324,ADM.N,BG.N,JBSS3.SA,MRFG3.SA,BEEF3.SA' # this will not work, they all need to be the same type

df_loans,e = ek.get_data("SCREEN(U(IN(DEALS)/*UNV:DEALSLOAN*/),IN(TR.LNParticipant(LNPartRole=LNBUP)," + instruments + "))", fields)
# this only returns the deals for 4296555324 (JBS)

NameError: name 'ek' is not defined

When putting different companies into the screener manually, it searches using PermID. In general, you should use PermID
since whilst RICs can change, PermIDs won't.

In [ ]:
## Simple export of LOAN deals for ADM, Cargill, JBS, Bunge, Minerva, Marfrig - not mapping to ultimate parents etc

instruments = '4295859830,4295859969,4295860302,4295903463,4296555324,5086635324'

fields = ["TR.LNSdcDealNum,TR.LNAnnouncementDate,TR.LNTrancheClosingDatePrint,TR.LNIssuer,TR.LNIssuerPERMID,TR.LNIssuerPriSIC,TR.LNIssuerNation,TR.LNTotalFacilityAmount(Scale=6),TR.LNTrancheAmount(Scale=6),TR.LNTargetMarket,TR.LNUseOfProceeds(Concat='|'),TR.LNTrancheType,TR.LNYieldType,TR.LNInitialPricingSpread(Concat='|'),TR.LNMaturityDate,TR.LNBookRunner(Concat='|'),TR.LNPrincipalAmountPerBookRunnerThisMarket(Scale=6),TR.LNMandatedArranger(Concat='|')"]

df_loans,e = ek.get_data("SCREEN(U(IN(DEALS)/*UNV:DEALSLOAN*/),IN(TR.LNParticipant(LNPartRole=LNBUP)," + instruments + "))", fields)
df_loans

df.to_csv("/Users/lydiamarsden/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Documents/Programming/Mini projects/eikon_api/loans_test.csv")


### Bond deals

Formula exported from Deal Screener:

=@TR("SCREEN(U(IN(DEALS)/*UNV:DEALSBOND*/),TR.NIisECM=False,IN(TR.NIParticipant(NIDealPartRole=IS,ISIP,ISUP),4295859830,4295859969,4295860302,4295903463,4296555324,5086635324))","TR.NISdcPackageNumber,TR.NISdcDealNumber,TR.NIIssueDate,TR.NITransactionStatus,TR.NIIssuer,TR.NIIssu"&"erPermID,TR.NIIssuerDomicileNation,TR.NIIssuerTRBCBusinessSector,TR.NIIssuerTRBCInd,TR.NIIssueType(C"&"oncat='|'),TR.NIProceedsAmtInclOverallotSoldThisMkt(Scale=6),TR.NIProceedsAmtInclOverallotSoldAllMkt"&"s(Scale=6),TR.NIGrossSpreadAsPctOfPrincipalAmtThisMktPrint,TR.NIGrossSpreadPerShareOrBondUsdPrint,TR"&".NITotalImputedDealFees(Scale=6),TR.NIBookRunner(Concat='|'),TR.NIBookRunnerCount,TR.NIManagers(Conc"&"at='|'),TR.NIAllManagersRole(Concat='|'),TR.NIManagerAllocShares(Concat='|'),TR.NIManagerAllocAmt(Sc"&"ale=6,Concat='|'),TR.NIIssuerUltParentPermID /*EncodedScreenStart[{""currency"":""USD"",""universeĊĀ"&"listsĊ[ĀnamĚċADM.NĐ""idĊ""4295903463įtypĨāompanĉđtooltipĳArchė-Dňiels-Mılňd CoįindexĊ0},ĤĦŃBGĭũĲċ508"&"6Ľ5324ĿŁŃcŅŇŉĿōŏőĳBēge Global SAũūŭĊ1űųħĳJBSS3.ƝŹĳĵķžĺ02ƅłĳƈņňŊ""ŌŎŐŒċƨSƜƞđŪŬŮ:2Ƥ""ĥƦċMRFGƬƮǈźĴĶ58ĸ8"&"30ƷƇƉƼƌǀƏǒarfrigƕƗƙƛFōdsǆƟǊĊ3ǎǐŵEEFǗǇİǚƱǝĸ969ǣƹǥƋŋƍǁĳšnėvaǹǈƠǋ4ǾŴưĶ65ȥƂƄǙȢȋȥƁƃȎċƺƊƽƿƎǂ""CǫgilƛIć (ƱȤ"&"Ȧƃ)Ǻơ:5}]đƚlě""acŐĖĊtrueȜɒɔĨfƚęđowȗĘhǩɟŞɚ""pǮmǫyOnlĉ:ɗəđɭĕatɞɠɫpubĝcɖɘɫƈēɗŁɮɰquoɽĊɩęű""ŭclusionġ:[ɌĒ"&"ĔĖĘeŧnɽxtĳDEALSBONďđŬƚPǫŐcőňtRŎŃIS,˅IPˇSUPŹTƆĊnuȼʖfȻɽĘĢʣƗȺcƚOłrɼʝʟTąeʠ˓ȼđmoŬĳsc˨nįţğGǮdTeņţʒ:ƥŃCăąʪƜ"&"˳eĆ ˼˾ɼĚŋˑċgǲƚ""}}]EncodedScreenEnd*/ ","Curn=USD CH=Fd")

In [7]:
## Simple export of BOND deals for ADM, JBS, Cargill, Bunge, Minerva, Marfrig

instruments = '4295859830,4295859969,4295860302,4295903463,4296555324,5086635324'

# to set currency, use {"Curn":"USD"}

fields = "TR.NISdcPackageNumber,TR.NISdcDealNumber,TR.NIIssueDate,TR.NITransactionStatus,TR.NIIssuer,TR.NIIssuerPermID,TR.NIIssuerDomicileNation,TR.NIIssuerTRBCBusinessSector,TR.NIIssuerTRBCInd,TR.NIIssueType(Concat='|'),TR.NIProceedsAmtInclOverallotSoldThisMkt(Scale=6),TR.NIProceedsAmtInclOverallotSoldAllMkts(Scale=6),TR.NIGrossSpreadAsPctOfPrincipalAmtThisMktPrint,TR.NIGrossSpreadPerShareOrBondUsdPrint,TR.NITotalImputedDealFees(Scale=6),TR.NIBookRunner(Concat='|'),TR.NIBookRunnerCount,TR.NIManagers(Concat='|'),TR.NIAllManagersRole(Concat='|'),TR.NIManagerAllocShares(Concat='|'),TR.NIManagerAllocAmt(Scale=6,Concat='|'),TR.NIIssuerUltParentPermID"

df,e = ek.get_data("SCREEN(U(IN(DEALS)/*UNV:DEALSBOND*/),TR.NIisECM=False,IN(TR.NIParticipant(NIDealPartRole=IS,ISIP,ISUP)," + instruments +"))", fields, {"Curn":"USD"})

NameError: name 'ek' is not defined

### Equity issuances

=@TR("SCREEN(U(IN(DEALS)/*UNV:DEALSBOND*/),TR.NIisECM=False,IN(TR.NIParticipant(NIDealPartRole=IS,ISIP,ISUP),4295859830,4295859969,4295860302,4295903463,4296555324,5086635324))","TR.NISdcPackageNumber,TR.NISdcDealNumber,TR.NIIssueDate,TR.NITransactionStatus,TR.NIIssuer,TR.NIIssu"&"erPermID,TR.NIIssuerDomicileNation,TR.NIIssuerTRBCBusinessSector,TR.NIIssuerTRBCInd,TR.NIIssueType(C"&"oncat='|'),TR.NIProceedsAmtInclOverallotSoldThisMkt(Scale=6),TR.NIProceedsAmtInclOverallotSoldAllMkt"&"s(Scale=6),TR.NIGrossSpreadAsPctOfPrincipalAmtThisMktPrint,TR.NIGrossSpreadPerShareOrBondUsdPrint,TR"&".NITotalImputedDealFees(Scale=6),TR.NIBookRunner(Concat='|'),TR.NIBookRunnerCount,TR.NIManagers(Conc"&"at='|'),TR.NIAllManagersRole(Concat='|'),TR.NIManagerAllocShares(Concat='|'),TR.NIManagerAllocAmt(Sc"&"ale=6,Concat='|'),TR.NIIssuerUltParentPermID /*EncodedScreenStart[{""currency"":""USD"",""universeĊĀ"&"listsĊ[ĀnamĚċADM.NĐ""idĊ""4295903463įtypĨāompanĉđtooltipĳArchė-Dňiels-Mılňd CoįindexĊ0},ĤĦŃBGĭũĲċ508"&"6Ľ5324ĿŁŃcŅŇŉĿōŏőĳBēge Global SAũūŭĊ1űųħĳJBSS3.ƝŹĳĵķžĺ02ƅłĳƈņňŊ""ŌŎŐŒċƨSƜƞđŪŬŮ:2Ƥ""ĥƦċMRFGƬƮǈźĴĶ58ĸ8"&"30ƷƇƉƼƌǀƏǒarfrigƕƗƙƛFōdsǆƟǊĊ3ǎǐŵEEFǗǇİǚƱǝĸ969ǣƹǥƋŋƍǁĳšnėvaǹǈƠǋ4ǾŴưĶ65ȥƂƄǙȢȋȥƁƃȎċƺƊƽƿƎǂ""CǫgilƛIć (ƱȤ"&"Ȧƃ)Ǻơ:5}]đƚlě""acŐĖĊtrueȜɒɔĨfƚęđowȗĘhǩɟŞɚ""pǮmǫyOnlĉ:ɗəđɭĕatɞɠɫpubĝcɖɘɫƈēɗŁɮɰquoɽĊɩęű""ŭclusionġ:[ɌĒ"&"ĔĖĘeŧnɽxtĳDEALSBONďđŬƚPǫŐcőňtRŎŃIS,˅IPˇSUPŹTƆĊnuȼʖfȻɽĘĢʣƗȺcƚOłrɼʝʟTąeʠ˓ȼđmoŬĳsc˨nįţğGǮdTeņţʒ:ƥŃCăąʪƜ"&"˳eĆ ˼˾ɼĚŋˑċgǲƚ""}}]EncodedScreenEnd*/ ","Curn=USD CH=Fd")

In [ ]:
## Simple export of equity issuances using Deal Screener template

instruments = '4295859830,4295859969,4295860302,4295903463,4296555324,5086635324'

fields = "TR.NISdcPackageNumber,TR.NISdcDealNumber,TR.NIIssueDate,TR.NITransactionStatus,TR.NIIssuer,TR.NIIssuerPermID,TR.NIIssuerDomicileNation,TR.NIIssuerTRBCBusinessSector,TR.NIIssuerTRBCInd,TR.NIIssueType(Concat='|'),TR.NIProceedsAmtInclOverallotSoldThisMkt(Scale=6),TR.NIProceedsAmtInclOverallotSoldAllMkts(Scale=6),TR.NIGrossSpreadAsPctOfPrincipalAmtThisMktPrint,TR.NIGrossSpreadPerShareOrBondUsdPrint,TR.NITotalImputedDealFees(Scale=6),TR.NIBookRunner(Concat='|'),TR.NIBookRunnerCount,TR.NIManagers(Concat='|'),TR.NIAllManagersRole(Concat='|'),TR.NIManagerAllocShares(Concat='|'),TR.NIManagerAllocAmt(Scale=6,Concat='|'),TR.NIIssuerUltParentPermID"

df_equity_issuances,e = ek.get_data("SCREEN(U(IN(DEALS)/*UNV:DEALSBOND*/),TR.NIisECM=False,IN(TR.NIParticipant(NIDealPartRole=IS,ISIP,ISUP)," + instruments + "))",fields,{"Curn":"USD"})
df_equity_issuances 

### Shareholders

ek.get_data

NOTE THAT FILES ARE EXPORTED AS UTF-8, so may appear strange in Mac for e.g., Spanish names

### Symbology

In [1]:
ek.get_symbology('4296555324', from_symbol_type='PermID', to_symbol_type='RIC')

NameError: name 'ek' is not defined